In [1]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [33]:
import torch
from PIL import Image
# setup device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
import requests

In [35]:
raw_image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

In [36]:
from lavis.models import load_model_and_preprocess

In [37]:
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip_feature_extractor", model_type="base", is_eval=True, device=device)

In [8]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [12]:
caption = "a large fountain spewing water into the air"
text_input = txt_processors["eval"](caption)

In [14]:
sample = {"image": image, "text_input": [text_input]}

In [15]:
features_multimodal = model.extract_features(sample)

In [16]:
features_image = model.extract_features(sample, mode="image")

In [19]:
features_image.image_embeds.shape

torch.Size([1, 197, 768])

In [20]:
print(features_image.image_embeds_proj.shape)

torch.Size([1, 197, 256])


In [23]:
features_image.image_embeds_proj[:,0,:].shape

torch.Size([1, 256])

In [24]:
model, vis_processors, txt_processors = load_model_and_preprocess("clip_feature_extractor", model_type="ViT-B-16", is_eval=True, device=device)


100%|████████████████████████████████████████| 351M/351M [00:02<00:00, 138MiB/s]


In [25]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
text = "merlion, a landmark in Singapore"

In [26]:

sample = {"image": image, "text_input": text}

clip_features = model.extract_features(sample)

image_features = clip_features.image_embeds_proj
text_features = clip_features.text_embeds_proj

In [28]:
image_features.shape

torch.Size([1, 512])

In [29]:
text_features.shape

torch.Size([1, 512])

In [31]:
cls_names = ["merlion",
"sky",
"giraffe",
"fountain",
"marina bay"]

In [32]:
sims = (image_features @ text_features.t())[0] / 0.01
probs = torch.nn.Softmax(dim=0)(sims).tolist()

for cls_nm, prob in zip(cls_names, probs):
    print(f"{cls_nm}: \t {prob:.3%}")

merlion: 	 100.000%
